In [1]:
import build_dataset

lfx = build_dataset.load_functions()
pfx = build_dataset.performance_functions()

pp = build_dataset.dataset_preprocess(local_activity_store='gc_activitydata_local.csv'
                                    ,local_activity_model_params='modeled_ef.csv')
lfx.metric_function_map

Local dataset available. process using `dataset_preprocess`


{'Daily_TSS': <bound method load_functions.daily_tss of <build_dataset.load_functions object at 0x7fa124414dc0>>,
 'TIZ2_5': <bound method load_functions.tiz2of5 of <build_dataset.load_functions object at 0x7fa124414dc0>>,
 'TIZ1_3': <bound method load_functions.tiz1of3 of <build_dataset.load_functions object at 0x7fa124414dc0>>,
 'TIZ3_3': <bound method load_functions.tiz3of3 of <build_dataset.load_functions object at 0x7fa124414dc0>>}

In [2]:
loaded_df = lfx.derive_load(frame=pp.activity_data, load_metric='TIZ1_3')
loaded_df = loaded_df[loaded_df['TIZ1_3'].cumsum() > 0].copy()

In [5]:
pfx.derive_performance(loaded_df, performance_metric='VO2')

NameError: name 'athlete_statics' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Data Processing

In [ ]:
data = data_original[data_original['Sport'] == 'Bike'].copy()
data['mod_pow_at_threshold'] = data['a'] + data['b']*athlete_statics['threshold_hr'] +  data['c']*(60*60)*20

data = data.groupby('date').agg({'L1_Time_in_Zone':'sum'
                                ,'L2_Time_in_Zone':'sum'
                                ,'L3_Time_in_Zone':'sum'
                                ,'L4_Time_in_Zone':'sum'
                                ,'L5_Time_in_Zone':'sum'
                                ,'L6_Time_in_Zone':'sum'
                                ,'L7_Time_in_Zone':'sum'
                                ,'mod_pow_at_threshold':'max'}).reset_index()

# data['date'] = data.index
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by=['date'])
data.index = pd.DatetimeIndex(data['date'])
missing_dates = pd.date_range(start=data.index.min(), end=data.index.max())
data = data.reindex(missing_dates, fill_value=0)
data['mod_pow_at_threshold'] = data['mod_pow_at_threshold'].replace(0,np.nan)
data['mod_pow_at_threshold'] = data['mod_pow_at_threshold'].fillna(method='ffill')
data = data.dropna()